In [ ]:
import json
import pandas as pd
import nltk
import numpy as np
import torch
!pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
train_df = pd.read_excel('drive/MyDrive/output.xlsx')
train_df.head()

,text,pos_tags,modality
0,RECITALS WHEREAS in connection with RFP ...,"[('RECITALS', 'NNP'), ('WHEREAS', 'NNP'), ('in...",P
1,The Recipient agrees to indemnify UNHCR in res...,"[('The', 'DT'), ('Recipient', 'NNP'), ('agrees...",O
2,Monthly virtual interactions need to be planne...,"[('Monthly', 'JJ'), ('virtual', 'JJ'), ('inter...",O
3,Notwithstanding the foregoing Mentor shall di...,"[('Notwithstanding', 'VBG'), ('the', 'DT'), ('...",P
4,The purpose of this Agreement is to permit eac...,"[('The', 'DT'), ('purpose', 'NN'), ('of', 'IN'...",P


In [ ]:
train_df["modality"].value_counts()

F    574
P    557
O    458
Name: modality, dtype: int64

In [ ]:
possible_labels = train_df.modality.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'P': 0, 'O': 1, 'F': 2}

In [ ]:
train_df['label'] = train_df.modality.replace(label_dict)

In [ ]:
up_train_df = train_df.drop(['modality'], axis=1)

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
!pip install datasets
from datasets import Dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.1 MB/s eta 0:00:00


In [ ]:
dataset = Dataset.from_pandas(up_train_df)
dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'pos_tags', 'label'],
        num_rows: 1430
    })
    test: Dataset({
        features: ['text', 'pos_tags', 'label'],
        num_rows: 159
    })
})


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1430 [00:00<?, ? examples/s]

Map:   0%|          | 0/159 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)
model_legal_bert = AutoModelForSequenceClassification.from_pretrained("nlpaueb/legal-bert-base-uncased", num_labels=3)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [ ]:
def compute_metrics(eval_pred):

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=10, learning_rate=1e-7, per_device_train_batch_size=4, per_device_eval_batch_size = 4)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1430))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(159))

In [ ]:
o=0
p=0
f=0

for i in small_eval_dataset["label"]:
  if(i==0):
    o+=1
  elif(i==1):
    p+=1
  else:
    f+=1

print(o)
print(p)
print(f)

63
39
57


In [ ]:
trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=small_train_dataset,

    eval_dataset=small_eval_dataset,

    compute_metrics=compute_metrics,

)

trainer_legal_bert = Trainer(

    model=model_legal_bert,

    args=training_args,

    train_dataset=small_train_dataset,

    eval_dataset=small_eval_dataset,

    compute_metrics=compute_metrics,

)

In [ ]:
import numpy as np
!pip install evaluate
import evaluate

metric = evaluate.load("accuracy")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.2 MB/s eta 0:00:00


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.080704,0.396226
2,1.126100,1.065065,0.396226
3,1.104500,1.053850,0.427673
4,1.104500,1.045011,0.490566
5,1.086400,1.036878,0.559748
6,1.082000,1.029913,0.553459
7,1.066300,1.023975,0.578616
8,1.066300,1.019637,0.591195
9,1.064400,1.016817,0.597484
10,1.050100,1.015968,0.597484


TrainOutput(global_step=3580, training_loss=1.08289627842397, metrics={'train_runtime': 1433.9868, 'train_samples_per_second': 9.972, 'train_steps_per_second': 2.497, 'total_flos': 3762521873510400.0, 'train_loss': 1.08289627842397, 'epoch': 10.0})

In [ ]:
trainer_legal_bert.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.090619,0.301887
2,1.121000,1.085492,0.308176
3,1.103900,1.083361,0.327044
4,1.103900,1.080952,0.358491
5,1.098400,1.078804,0.383648
6,1.088500,1.077063,0.389937
7,1.084400,1.075809,0.377358
8,1.084400,1.074704,0.383648
9,1.084200,1.074194,0.396226
10,1.080000,1.074061,0.396226


TrainOutput(global_step=3580, training_loss=1.0939192532161095, metrics={'train_runtime': 1433.169, 'train_samples_per_second': 9.978, 'train_steps_per_second': 2.498, 'total_flos': 3762521873510400.0, 'train_loss': 1.0939192532161095, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 1.0159682035446167,
 'eval_accuracy': 0.5974842767295597,
 'eval_runtime': 4.9171,
 'eval_samples_per_second': 32.336,
 'eval_steps_per_second': 8.135,
 'epoch': 10.0}

In [ ]:
trainer_legal_bert.evaluate()

{'eval_loss': 1.074061393737793,
 'eval_accuracy': 0.39622641509433965,
 'eval_runtime': 5.2304,
 'eval_samples_per_second': 30.399,
 'eval_steps_per_second': 7.648,
 'epoch': 10.0}

In [ ]:
trainer. save_model("drive/MyDrive/test-bert")

In [ ]:
trainer_legal_bert.save_model("drive/MyDrive/test-legal-bert")

### Model testing using generated data

In [ ]:
ext_test_df = pd.read_excel('drive/MyDrive/ext_test.xlsx')
ext_test_df = ext_test_df.drop(["label"], axis=1)
l = ext_test_df["text"].values.tolist()
len(l)

58

In [ ]:
# Load Pre-trained model

In [ ]:
from transformers import AutoTokenizer, AutoModel
test_model = AutoModel.from_pretrained('drive/MyDrive/test-bert/')
test_legal_model = AutoModel.from_pretrained('drive/MyDrive/test-legal-bert/')

Some weights of the model checkpoint at drive/MyDrive/test-bert/ were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at drive/MyDrive/test-legal-bert/ were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT e

In [ ]:
dataset = Dataset.from_pandas(ext_test_df)

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,return_tensors="pt")
test_tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/58 [00:00<?, ? examples/s]

In [ ]:
test_tokenized_datasets

In [ ]:
sentences = l
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tokenize the input sentences
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)

# Pass the inputs through the model
outputs = model(**inputs)

# Get the predicted probabilities for each class
probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

# Get the predicted label for each input sentence
preds = torch.argmax(probs, dim=-1)

# Print the predicted labels and probabilities for each input sentence
for i, sentence in enumerate(sentences):
    print(f"Sentence {i+1}: {sentence}")
    print(f"Predicted label: {preds[i]}")
    print(f"Predicted probabilities: {probs[i]}\n")

Sentence 1: The receiving party shall maintain the confidentiality of the shared data and shall not disclose it to any third party.
Predicted label: 0
Predicted probabilities: tensor([0.3927, 0.2218, 0.3855], device='cuda:0', grad_fn=<SelectBackward0>)

Sentence 2: The receiving party shall use the shared data only for the purpose of the agreement and shall not use it for any other purpose without the express written consent of the disclosing party.
Predicted label: 2
Predicted probabilities: tensor([0.3791, 0.2229, 0.3979], device='cuda:0', grad_fn=<SelectBackward0>)

Sentence 3: The receiving party shall implement appropriate security measures to protect the shared data from unauthorized access or disclosure.
Predicted label: 0
Predicted probabilities: tensor([0.4200, 0.2345, 0.3455], device='cuda:0', grad_fn=<SelectBackward0>)

Sentence 4: The receiving party shall promptly notify the disclosing party of any suspected or actual breach of the security of the shared data.
Predicted la